# OpenSpiel: Deep CFR in PyTorch
In this notebook we present an example of how to use OpenSpiel with PyTorch as the learning library. This is possible since most of the components in OpenSpiel are agnostic of the learning library used.

In this notebook we implement the Deep Counterfactual Regret algorithm in PyTorch on the game of Kuhn Poker. Adapted from [this example](https://github.com/deepmind/open_spiel/blob/master/open_spiel/python/examples/deep_cfr.py).


## Build and Install OpenSpiel
Clone the latest version of OpenSpiel from Github.

In [0]:
#@title
%cd /usr/local
!git clone https://github.com/deepmind/open_spiel;
%cd open_spiel

Install the requirements and build the library.

In [0]:
#@title
!./install.sh
!pip install -r requirements.txt;
!mkdir -p /usr/local/open_spiel/build
%cd /usr/local/open_spiel/build
!CXX=g++ cmake -DPython_TARGET_VERSION=3.6 -DCMAKE_CXX_COMPILER=${CXX} ../open_spiel;
!make -j$(nproc);

Set up the PYTHONPATH to enable the Python bindings for the current instance.

In [0]:
#@title
%cd /content
import sys
sys.path.append('/usr/local/open_spiel')
sys.path.append('/usr/local/open_spiel/build/python')

## Import dependencies

In [0]:
from open_spiel.python import policy
from open_spiel.python.algorithms import exploitability
from open_spiel.python.algorithms.deep_cfr import AdvantageMemory, StrategyMemory, FixedSizeRingBuffer
import pyspiel
from absl import logging
import collections
import numpy as np

import torch
import torch.nn as nn

In [0]:
def reset_params(m):
  if isinstance(m, nn.Linear):
    m.reset_parameters()

# Defining fully-connected network to use as policy and advantage network
class MLP(nn.Module):
  def __init__(self, layers):
    """
      Creates a MLP with hidden units defined by `layers`.
    """
    super(MLP, self).__init__()
    layers = [nn.Linear(layers[i], layers[i+1]) for i in range(len(layers[:-1]))]
    self._model = nn.Sequential(*layers)
    
  def forward(self, input):
    return self._model(input)
  
  def reset(self):
    self._model.apply(reset_params)

## Implement DeepCFR
We follow the structure in the [Tensorflow implementation](https://github.com/deepmind/open_spiel/blob/master/open_spiel/python/algorithms/deep_cfr.py), rewiting the optimization steps in PyTorch. 

In [0]:
class DeepCFR(policy.Policy):
  def __init__(self,
               game,
               policy_network_layers=(256, 256),
               advantage_network_layers=(128, 128),
               num_iterations=100,
               num_traversals=20,
               learning_rate=1e-4,
               batch_size_advantage=None,
               batch_size_strategy=None,
               memory_capacity=int(1e6)):
    all_players = list(range(game.num_players()))
    super(DeepCFR, self).__init__(game, all_players)
    self._game = game
    self._batch_size_advantage = batch_size_advantage
    self._batch_size_strategy = batch_size_strategy
    self._num_players = game.num_players()
    self._root_node = self._game.new_initial_state()
    self._embedding_size = len(
        self._root_node.information_state_tensor(0))
    self._num_iterations = num_iterations
    self._num_traversals = num_traversals
    self._num_actions = game.num_distinct_actions()
    self._iteration = 1


    # Define strategy network, loss & memory.
    self._strategy_memories = FixedSizeRingBuffer(memory_capacity)
    self._policy_network = MLP([self._embedding_size] + list(policy_network_layers) + [self._num_actions])
    # Illegal actions are handled in the traversal code where expected payoff
    # and sampled regret is computed from the advantage networks.
    self._policy_sm = nn.Softmax(dim=-1)
    self._loss_policy = nn.MSELoss()
    self._optimizer_policy = torch.optim.Adam(self._policy_network.parameters(), lr=learning_rate)


    # Define advantage network, loss & memory. (One per player)
    self._advantage_memories = [
        FixedSizeRingBuffer(memory_capacity) for _ in range(self._num_players)
    ]
    self._advantage_networks = [
        MLP([self._embedding_size] + list(advantage_network_layers) + [self._num_actions])
        for _ in range(self._num_players)
    ]
    self._optimizer_advantages = []
    for p in range(self._num_players):
      self._loss_advantages = nn.MSELoss(reduction='mean')
      self._optimizer_advantages.append(
          torch.optim.Adam(self._advantage_networks[p].parameters(), lr=learning_rate))

  @property
  def advantage_buffers(self):
    return self._advantage_memories

  @property
  def strategy_buffer(self):
    return self._strategy_memories

  def clear_advantage_buffers(self):
    for p in range(self._num_players):
      self._advantage_memories[p].clear()

  def reinitialize_advantage_networks(self):
    for p in range(self._num_players):
      self._advantage_networks[p].reset()

  def solve(self):
    """Solution logic for Deep CFR."""
    advantage_losses = collections.defaultdict(list)
    for _ in range(self._num_iterations):
      for p in range(self._num_players):
        for _ in range(self._num_traversals):
          self._traverse_game_tree(self._root_node, p)
        self.reinitialize_advantage_networks()
        # Re-initialize advantage networks and train from scratch.
        advantage_losses[p].append(self._learn_advantage_network(p))
      self._iteration += 1
    # Train policy network.
    policy_loss = self._learn_strategy_network()
    return self._policy_network, advantage_losses, policy_loss

  def _traverse_game_tree(self, state, player):
    expected_payoff = collections.defaultdict(float)
    if state.is_terminal():
      # Terminal state get returns.
      return state.returns()[player]
    elif state.is_chance_node():
      # If this is a chance node, sample an action
      action = np.random.choice([i[0] for i in state.chance_outcomes()])
      return self._traverse_game_tree(state.child(action), player)
    elif state.current_player() == player:
      sampled_regret = collections.defaultdict(float)
      # Update the policy over the info set & actions via regret matching.
      advantages, strategy = self._sample_action_from_advantage(state, player)
      for action in state.legal_actions():
        expected_payoff[action] = self._traverse_game_tree(
            state.child(action), player)
      for action in state.legal_actions():
        sampled_regret[action] = expected_payoff[action]
        for a_ in state.legal_actions():
          sampled_regret[action] -= strategy[a_] * expected_payoff[a_]
        self._advantage_memories[player].add(
            AdvantageMemory(state.information_state_tensor(),
                            self._iteration, advantages, action))
      return max(expected_payoff.values())
    else:
      other_player = state.current_player()
      _, strategy = self._sample_action_from_advantage(state, other_player)
      # Recompute distribution dor numerical errors.
      probs = np.array(strategy)
      probs /= probs.sum()
      sampled_action = np.random.choice(range(self._num_actions), p=probs)
      self._strategy_memories.add(
          StrategyMemory(
              state.information_state_tensor(other_player),
              self._iteration, strategy))
      return self._traverse_game_tree(state.child(sampled_action), player)

  def _sample_action_from_advantage(self, state, player):
    """Returns an info state policy by applying regret-matching.
    Args:
      state: Current OpenSpiel game state.
      player: (int) Player index over which to compute regrets.
    Returns:
      1. (list) Advantage values for info state actions indexed by action.
      2. (list) Matched regrets, prob for actions indexed by action.
    """
    info_state = state.information_state_tensor(player)
    legal_actions = state.legal_actions(player)
    advantages = self._advantage_networks[player](torch.FloatTensor(np.expand_dims(info_state, axis=0)))[0].detach().numpy()
    advantages = [max(0., advantage) for advantage in advantages]
    cumulative_regret = np.sum([advantages[action] for action in legal_actions])
    matched_regrets = np.array([0.] * self._num_actions)
    for action in legal_actions:
      if cumulative_regret > 0.:
        matched_regrets[action] = advantages[action] / cumulative_regret
      else:
        matched_regrets[action] = 1 / self._num_actions
    return advantages, matched_regrets

  def _learn_advantage_network(self, player):
    """Compute the loss on sampled transitions and perform a Q-network update.
    If there are not enough elements in the buffer, no loss is computed and
    `None` is returned instead.
    Args:
      player: (int) player index.
    Returns:
      The average loss over the advantage network.
    """
    if self._batch_size_advantage:
      samples = self._advantage_memories[player].sample(
          self._batch_size_advantage)
    else:
      samples = self._advantage_memories[player]
    info_states = []
    advantages = []
    iterations = []
    for s in samples:
      info_states.append(s.info_state)
      advantages.append(s.advantage)
      iterations.append([s.iteration])
    # Ensure some samples have been gathered.
    if not info_states:
      return None
    advantages = torch.FloatTensor(np.array(advantages))
    iters = torch.FloatTensor(np.sqrt(np.array(iterations)))
    outputs = self._advantage_networks[player](torch.FloatTensor(np.array(info_states)))
    self._optimizer_advantages[player].zero_grad()
    loss_advantages = self._loss_advantages(iters * outputs, iters * advantages)
    loss_advantages.backward()
    self._optimizer_advantages[player].step()
    return loss_advantages.detach().numpy()

  def _learn_strategy_network(self):
    """Compute the loss over the strategy network.
    Returns:
      The average loss obtained on this batch of transitions or `None`.
    """
    if self._batch_size_strategy:
      samples = self._strategy_memories.sample(self._batch_size_strategy)
    else:
      samples = self._strategy_memories
    info_states = []
    action_probs = []
    iterations = []
    for s in samples:
      info_states.append(s.info_state)
      action_probs.append(s.strategy_action_probs)
      iterations.append([s.iteration])

    iters = torch.FloatTensor(np.sqrt(np.array(iterations)))
    ac_probs = torch.FloatTensor(np.array(np.squeeze(action_probs)))
    outputs = self._policy_sm(self._policy_network(torch.FloatTensor(np.array(info_states))))
    self._optimizer_policy.zero_grad()
    loss_strategy = self._loss_policy(iters * outputs, iters * ac_probs)
    loss_strategy.backward()
    self._optimizer_policy.step()
    return loss_strategy.detach().numpy()

  def action_probabilities(self, state):
    """Returns action probabilities dict for a single batch."""
    info_state_vector = np.array(state.information_state_tensor())
    if len(info_state_vector.shape) == 1:
      info_state_vector = np.expand_dims(info_state_vector, axis=0)
    probs = self._policy_sm(self._policy_network(torch.FloatTensor(info_state_vector))).detach().numpy()
    return {i: probs[0][i] for i in range(self._num_actions)}


## Train the model

In [0]:
game = pyspiel.load_game('kuhn_poker')

In [0]:
solver = DeepCFR(game,
      policy_network_layers=(32, 32),
      advantage_network_layers=(16, 16),
      num_iterations=400,
      num_traversals=40,
      learning_rate=1e-3,
      batch_size_advantage=None,
      batch_size_strategy=None,
      memory_capacity=1e7)

In [0]:
_, advantage_losses, policy_loss = solver.solve()

In [0]:
for player, losses in list(advantage_losses.items()):
  print("Advantage for player:", player,
                losses[:2] + ["..."] + losses[-2:])
  print("Advantage Buffer Size for player", player,
                len(solver.advantage_buffers[player]))
print("Strategy Buffer Size:",
              len(solver.strategy_buffer))
print("Final policy loss:", policy_loss)
conv = exploitability.nash_conv(
    game,
    policy.tabular_policy_from_callable(game, solver.action_probabilities))
print("Deep CFR - NashConv:", conv)